In [18]:
from collections import OrderedDict
from keras import utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import (
    Activation,
    Dense,
    Dropout,
    Flatten, 
    Conv2D,
    MaxPooling2D)
import numpy as np
import pandas as pd
import tensorflow as tf

In [157]:
# Data structure :'Word POS_tag syntactic_chunk_tag named_entity_tag \n'
# Order a word = prefixe 2, 3, 4, 5 then suffixe 2, 3, 4, 5, capitals, numbers, middle_dashes, appostrophies

# train = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset_CONLL\eng.train', 'at')
# testa = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset_CONLL\eng.testa')
# testb = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset_CONLL\eng.testb')

length_max_sentence = 113
  
def get_words(path): # Add a 'r' before the path for raw string, pos between 0 and 3 see data structure
    data = open(path)
    list_words = []
    new_data = data.readlines()[2:] # Skip first 2 lines
    mylist = list(new_data) # Make the data readable
    for i in range(500): # TODO : CHANGES number TO len(mylist)
        mot = splitter(mylist[i], ' ', 0, 'getwords')
        if mot: # it return None if the line est empty
            list_words.append(mot)
    return list_words

def get_sentences(path):
    data = open(path)
    list_sentences = []
    list_entities = []
    new_data = data.readlines()[2:] # Skip first 2 lines
    mylist = list(new_data) # Make the data readable
    entity_of_sentence = []
    sentence = []
    for i in range(500): # TODO : CHANGES number TO len(mylist)
        mot = splitter(mylist[i], ' ', 0, 'getsentences')
        entity = splitter(mylist[i], ' ', 3, 'getsentences')
        if mot == '\n': # TODO: Check for the last!! 
            list_sentences.append(sentence)
            sentence = []
            list_entities.append(entity_of_sentence)
            entity_of_sentence = []
        else:
            sentence.append(mot)
            entity_of_sentence.append(entity)
#     print('LIST SENTENCES', list_sentences)
    return(list_sentences, list_entities)


TODO : Il y a un probleme chelou je pense que ma fonction splitter a une faute avec les \n (il y a des \n dans les entities chelou)
def splitter(line, separator, pos, function_calling): # pos between 0 and 3 for CONLL2003
    if line.isspace() and function_calling == 'getwords': # Removing blank lines because we don't want blank line in list_of_words
        return
    new_sentence = []
    for word in line.split(separator):
        new_sentence.append(word)
    print(new_sentence, len(new_sentence))
    if len(new_sentence) == 1: # this correspond to a new line (\n)
        return
    else:
        return new_sentence[pos]

def get_dictionary(list_of_words): # Return a dict with how many times a word appears
    dicti = {}
    for word in list_of_words:
        if word in dicti:
            dicti[word] = dicti[word] + 1
        else:
            dicti[word] = 1
    return dicti

# We only take into consideration the words that appears a certain number of times, and not the prefixes and suffixes
# that appears a certain number of time
def get_reduced_dictionary(dicti, limit): # return a dict with key >= limit
    for v, k in list(dicti.items()):
        if k <= limit: # We don't want to take into account words that doesn't appear enough (limit)
            del dicti[v]
    return dicti




# INUTILE JE PENSE - a verifier
# def indexation_dict(dicti): # Attention : local modification of the dictionnary
#     # dicti must be sorted beforehand
#     index = 0
#     for k in dicti.keys():
#         dicti[k] = index
#         index += 1 
#     return dicti



def get_prefixe(reduced_dict, prefixe_length): # Return an orderer dictionnary with index as values
    #TODO : remove all ponctuations signs
    dict_prefixes = {}
    index = 0
    for word in reduced_dict:
        prefixe = word[:prefixe_length]
        if prefixe in dict_prefixes:
            continue
        else:
            dict_prefixes[prefixe] = index
            index += 1 
    ordered_dict_prefixes = OrderedDict(sorted(dict_prefixes.items(), key=lambda t: t[1])) # Ordered by index
#     print('prefixes', ordered_dict_prefixes)
    return ordered_dict_prefixes
    
#     list_prefixes = []
#     for word in reduced_dict:
#         prefixe = word[:prefixe_length]
#         list_prefixes.append(prefixe)
#     return list_prefixes


def get_suffixe(reduced_dict, suffixe_length):
    #TODO : remove all ponctuations signs
    dict_suffixes = {}
    index = 0
    for word in reduced_dict:
        util_position = suffixe_length * (-1)
        suffixe = word[util_position:]
        if suffixe in dict_suffixes:
            continue
        else:
            dict_suffixes[suffixe] = index
            index += 1
    ordered_dict_suffixes = OrderedDict(sorted(dict_suffixes.items(), key=lambda t: t[1])) # Ordered by index
#     print('suffixes', ordered_dict_suffixes)
    return ordered_dict_suffixes
    
    
#     list_suffixes = []
#     for word in reduced_dict:
#         util_position = suffixe_length * (-1)
#         suffixe = word[util_position:]
#         list_suffixes.append(suffixe)
#     return list_suffixes


def has_capital(string):
    if any(char.isupper() for char in string):
        return 1
    return 0


def has_numbers(string):
    if any(char.isdigit() for char in string):
        return 1
    return 0


def has_appostrophy(string):
    if any(char == '\'' for char in string):
        return 1
    return 0

def has_middle_dash(string):
    if any(char == '-' for char in string):
        return 1
    return 0

# Now we need to create a vector to describe the words

def get_vector_for_prefixe(word, prefixe_length, dict_prefixe):
    # A word has one prefixe maximum
    vector_prefixe = [0] * len(dict_prefixe)
    prefixe = word[:prefixe_length]
    if prefixe in dict_prefixe.keys():
        vector_prefixe[dict_prefixe[prefixe]] = 1
#     print('vector prefixe', vector_prefixe)
    return vector_prefixe

def get_vector_for_suffixe(word, suffixe_length, dict_suffixe):
    vector_suffixe = [0] * len(dict_suffixe)
    util_position = suffixe_length * (-1)
    suffixe = word[util_position:]
    if suffixe in dict_suffixe.keys():
        vector_suffixe[dict_suffixe[suffixe]] = 1
#     print('vector suffixe', vector_suffixevector_suffixe)
    return vector_suffixe



def get_array_for_sentence(sentence, dict_prefixe_2, dict_prefixe_3, dict_prefixe_4, dict_prefixe_5,
                          dict_suffixe_2, dict_suffixe_3, dict_suffixe_4, dict_suffixe_5):
#     output = np.array([[]])
    number_column = (len(dict_prefixe_2) + len(dict_prefixe_3) + len(dict_prefixe_4) + len(dict_prefixe_5) +
                    len(dict_suffixe_2) + len(dict_suffixe_3) + len(dict_suffixe_4) +len(dict_suffixe_5) + 4)
    output = np.zeros((length_max_sentence, number_column)) # this way we don't need to add empty words after
    index = 0
    for word in sentence: 
        word_description = get_vector_for_prefixe(word, 2, dict_prefixe_2)
        word_description = word_description + get_vector_for_prefixe(word, 3, dict_prefixe_3)
        word_description = word_description + get_vector_for_prefixe(word, 4, dict_prefixe_4)
        word_description = word_description + get_vector_for_prefixe(word, 5, dict_prefixe_5)
        word_description = word_description + get_vector_for_suffixe(word, 2, dict_suffixe_2)
        word_description = word_description + get_vector_for_suffixe(word, 3, dict_suffixe_3)
        word_description = word_description + get_vector_for_suffixe(word, 4, dict_suffixe_4)
        word_description = word_description + get_vector_for_suffixe(word, 5, dict_suffixe_5)
        word_description = word_description + [has_capital(word)]
        word_description = word_description + [has_numbers(word)]
        word_description = word_description + [has_middle_dash(word)]
        word_description = word_description + [has_appostrophy(word)]
#         print('word', word_description)
        output[index] = word_description
        index += 1
    print('L\'input du NN est de taille', len(word_description))
    print(output, output.shape)
    return output

def get_features():
    list_of_words = get_words(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset_CONLL\eng.train')
    list_of_sentences, list_of_entities = get_sentences(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset_CONLL\eng.train')
#     print(list_of_words)
    dicti = get_dictionary(list_of_words)
    reduced_dict = get_reduced_dictionary(dicti, 5)
    dict_prefixe_2 = get_prefixe(reduced_dict, 2)
    dict_prefixe_3 = get_prefixe(reduced_dict, 3)
    dict_prefixe_4 = get_prefixe(reduced_dict, 4)
    dict_prefixe_5 = get_prefixe(reduced_dict, 5)
    dict_suffixe_2 = get_suffixe(reduced_dict, 2)
    dict_suffixe_3 = get_suffixe(reduced_dict, 3)
    dict_suffixe_4 = get_suffixe(reduced_dict, 4)
    dict_suffixe_5 = get_suffixe(reduced_dict, 5)
    print(list_of_sentences[0])
    get_array_for_sentence(list_of_sentences[0], dict_prefixe_2, dict_prefixe_3, dict_prefixe_4, dict_prefixe_5,
                           dict_suffixe_2, dict_suffixe_3, dict_suffixe_4, dict_suffixe_5)

    
  
get_features()  
#TODO REMOVE ALL DOCSTART LINES    


['EU', 'NNP', 'I-NP', 'I-ORG\n'] 4
['rejects', 'VBZ', 'I-VP', 'O\n'] 4
['German', 'JJ', 'I-NP', 'I-MISC\n'] 4
['call', 'NN', 'I-NP', 'O\n'] 4
['to', 'TO', 'I-VP', 'O\n'] 4
['boycott', 'VB', 'I-VP', 'O\n'] 4
['British', 'JJ', 'I-NP', 'I-MISC\n'] 4
['lamb', 'NN', 'I-NP', 'O\n'] 4
['.', '.', 'O', 'O\n'] 4
['Peter', 'NNP', 'I-NP', 'I-PER\n'] 4
['Blackburn', 'NNP', 'I-NP', 'I-PER\n'] 4
['BRUSSELS', 'NNP', 'I-NP', 'I-LOC\n'] 4
['1996-08-22', 'CD', 'I-NP', 'O\n'] 4
['The', 'DT', 'I-NP', 'O\n'] 4
['European', 'NNP', 'I-NP', 'I-ORG\n'] 4
['Commission', 'NNP', 'I-NP', 'I-ORG\n'] 4
['said', 'VBD', 'I-VP', 'O\n'] 4
['on', 'IN', 'I-PP', 'O\n'] 4
['Thursday', 'NNP', 'I-NP', 'O\n'] 4
['it', 'PRP', 'B-NP', 'O\n'] 4
['disagreed', 'VBD', 'I-VP', 'O\n'] 4
['with', 'IN', 'I-PP', 'O\n'] 4
['German', 'JJ', 'I-NP', 'I-MISC\n'] 4
['advice', 'NN', 'I-NP', 'O\n'] 4
['to', 'TO', 'I-PP', 'O\n'] 4
['consumers', 'NNS', 'I-NP', 'O\n'] 4
['to', 'TO', 'I-VP', 'O\n'] 4
['shun', 'VB', 'I-VP', 'O\n'] 4
['British', 'JJ', 

['illness', 'NN', 'I-NP', 'O\n'] 4
['illness', 'NN', 'I-NP', 'O\n'] 4
['similar', 'JJ', 'I-ADJP', 'O\n'] 4
['similar', 'JJ', 'I-ADJP', 'O\n'] 4
['to', 'TO', 'I-ADJP', 'O\n'] 4
['to', 'TO', 'I-ADJP', 'O\n'] 4
['mad', 'JJ', 'I-ADJP', 'O\n'] 4
['mad', 'JJ', 'I-ADJP', 'O\n'] 4
['cow', 'NN', 'I-NP', 'O\n'] 4
['cow', 'NN', 'I-NP', 'O\n'] 4
['disease', 'NN', 'I-NP', 'O\n'] 4
['disease', 'NN', 'I-NP', 'O\n'] 4
['by', 'IN', 'I-PP', 'O\n'] 4
['by', 'IN', 'I-PP', 'O\n'] 4
['eating', 'VBG', 'I-VP', 'O\n'] 4
['eating', 'VBG', 'I-VP', 'O\n'] 4
['contaminated', 'VBN', 'I-VP', 'O\n'] 4
['contaminated', 'VBN', 'I-VP', 'O\n'] 4
['beef', 'NN', 'I-NP', 'O\n'] 4
['beef', 'NN', 'I-NP', 'O\n'] 4
['.', '.', 'O', 'O\n'] 4
['.', '.', 'O', 'O\n'] 4
['\n'] 1
['\n'] 1
['Germany', 'NNP', 'I-NP', 'I-LOC\n'] 4
['Germany', 'NNP', 'I-NP', 'I-LOC\n'] 4
['imported', 'VBD', 'I-VP', 'O\n'] 4
['imported', 'VBD', 'I-VP', 'O\n'] 4
['47,600', 'CD', 'I-NP', 'O\n'] 4
['47,600', 'CD', 'I-NP', 'O\n'] 4
['sheep', 'NN', 'I-NP', 'O\n

IndexError: list index out of range

In [71]:
test={
    "date": 1999,
    "bojour": 3, 
    "popo": 15,
    "bojour2" : 100
}
if "date" in test.keys():
    print('pap')

pap


In [154]:
# m = np.zeros((length_max_sentence, len(word_description)))
liste = [1, 2, 3]
liste2 = [4, 5, 6]

print(not liste[4])


IndexError: list index out of range

In [ ]:
"""
def get_capital(list_words):
    # This function returns 2 lists:
    list_capitals = []   # 1/ If the word have a capital
    list_first_word = [] # 2/ If the word is the 1st word of the sentence
    chara_end_sentence = ['.', '!', '?', '...'] # TODO : Check for '...' if it should be in this list
    flag_new_sentence = True
    for word in list_words:
        letter = word[0] # First 'letter' of the word; can be number or special chara
        if letter.isupper():
            list_capitals.append(1)
            if flag_new_sentence:
                list_first_word.append(1)
            else:
                list_first_word.append(0)
        else:
            list_capitals.append(0)
            list_first_word.append(0)
        if letter in chara_end_sentence:
            flag_new_sentence = True
            continue
        flag_new_sentence = False
#     print(list_capitals, list_first_word)
    return (list_capitals, list_first_word)



def get_number(list_words): # Return array if there is a number in the word
    list_numbers = []
    for word in list_words:
        if has_numbers(word):
            list_numbers.append(1)
        else:
            list_numbers.append(0)
#     print(list_numbers)
    return list_numbers



def get_middle_dash(list_words):
    list_middle_dashes = []
    for word in list_words:
        if has_middle_dash(word):
            list_middle_dashes.append(1)
        else:
            list_middle_dashes.append(0)
#     print(list_middle_dashes)
    return list_middle_dashes


    
def get_appostrophy(list_words):
    list_appostrophy = []
    for word in list_words:
        if has_appostrophy(word):
            list_appostrophy.append(1)
        else:
            list_appostrophy.append(0)
#     print(list_appostrophy)
    return list_appostrophy

def get_vector_for_prefixe(word, prefixe_length, list_prefixe):
    vector_prefixe = [] # It should be of size get_prefixe()
    prefixe = word[:prefixe_length]
    for pref in list_prefixe:
        if pref == prefixe:
            vector_prefixe.append(1)
        else:
            vector_prefixe.append(0)
#     print('vector prefixe', vector_prefixe)
    return vector_prefixe



def get_vector_for_suffixe(word, suffixe_length, list_suffixe):
    vector_suffixe = []
    util_position = suffixe_length * (-1)
    suffixe = word[util_position:]
    for suf in list_suffixe:
        if suf == suffixe:
            vector_suffixe.append(1)
        else:
            vector_suffixe.append(0)
#     print('vector suffixe', vector_suffixe)
    return vector_suffixe

"""